In [1]:
import os
import re
import xml.etree.ElementTree as ET

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [2]:

def extract_data_from_xml(content):
    root = ET.fromstring(content)
    
    performance_estimates = root.find("PerformanceEstimates")
    average_case_latency = performance_estimates.find("SummaryOfOverallLatency").find("Average-caseLatency").text

    area_estimates = root.find("AreaEstimates")
    resources = area_estimates.find("Resources")
    bram = resources.find("BRAM_18K").text
    dsp = resources.find("DSP").text
    ff = resources.find("FF").text
    lut = resources.find("LUT").text
    uram = resources.find("URAM").text

    return [average_case_latency, bram, dsp, ff, lut, uram]

def extract_data_from_pwr(content):
    on_chip_power = re.search(r'Total On-Chip Power \(W\)\s*\|\s*(\d+\.\d+)', content).group(1)
    fpga_power = re.search(r'FPGA Power \(W\)\s*\|\s*(\d+\.\d+)', content).group(1)
    hbm_power = re.search(r'HBM Power \(W\)\s*\|\s*(\d+\.\d+)', content).group(1)
    dynamic_power = re.search(r'Dynamic \(W\)\s*\|\s*(\d+\.\d+)', content).group(1)

    return [on_chip_power, fpga_power, hbm_power, dynamic_power]

path_to_files = "./auto_test_results"  # Current directory
results_dict = {}

for filename in os.listdir(path_to_files):
    if filename.startswith("uut_top_csynth_"):
        name_without_ext = os.path.splitext(filename)[0]
        numbers = re.findall(r'\d+', filename)
        
        # Handle XML files
        if filename.endswith(".xml"):
            with open(os.path.join(path_to_files, filename), 'r') as f:
                content = f.read()
                xml_results = extract_data_from_xml(content)
                if name_without_ext not in results_dict:
                    results_dict[name_without_ext] = [filename, numbers, xml_results]
                else:
                    results_dict[name_without_ext].extend([xml_results])
                    
for filename in os.listdir(path_to_files):
    if filename.startswith("uut_top_csynth_"):
        name_without_ext = os.path.splitext(filename)[0]
        numbers = re.findall(r'\d+', filename)
        # Handle PWR files
        if filename.endswith(".pwr"):
            with open(os.path.join(path_to_files, filename), 'r') as f:
                content = f.read()
                pwr_results = extract_data_from_pwr(content)
                if name_without_ext not in results_dict:
                    results_dict[name_without_ext] = [filename, numbers, pwr_results]
                else:
                    results_dict[name_without_ext].extend([pwr_results])

# Convert dictionary to list of results
results = list(results_dict.values())

for i in range(5):
    print(results[i])
#[Name,[inH,inW,nChan,fH,fW,nFilt,outH,outW],[latency,BRAM,DSP,FF,LUT,URAM],[TotalPower,FPGAPower,HBMPower,DynamicPower]]

['uut_top_csynth_inH128_inW32_nChan2_fH4_fW4_nFilt2_outH10_outW10.xml', ['128', '32', '2', '4', '4', '2', '10', '10'], ['432', '0', '190', '156303', '96933', '0'], ['5.315', '5.072', '0.243', '2.143']]
['uut_top_csynth_inH32_inW64_nChan2_fH4_fW2_nFilt2_outH10_outW10.xml', ['32', '64', '2', '4', '2', '2', '10', '10'], ['254', '0', '115', '55807', '52107', '0'], ['3.899', '3.658', '0.241', '0.756']]
['uut_top_csynth_inH32_inW32_nChan2_fH2_fW2_nFilt4_outH10_outW10.xml', ['32', '32', '2', '2', '2', '4', '10', '10'], ['391', '0', '80', '53030', '38165', '0'], ['5.454', '5.212', '0.243', '2.279']]
['uut_top_csynth_inH10_inW10_nChan1_fH1_fW1_nFilt1_outH10_outW10.xml', ['10', '10', '1', '1', '1', '1', '10', '10'], ['62', '0', '10', '1841', '3028', '0'], ['3.164', '2.924', '0.241', '0.036']]
['uut_top_csynth_inH64_inW64_nChan2_fH4_fW4_nFilt2_outH10_outW10.xml', ['64', '64', '2', '4', '4', '2', '10', '10'], ['432', '0', '190', '156303', '96933', '0'], ['5.203', '4.961', '0.243', '2.033']]


In [6]:
X = [list(map(int, result[1])) for result in results]
y_area = [list(map(int, result[2])) for result in results]
y_power = [list(map(float, result[3])) for result in results]

# Concatenate area and power data to form a combined Y
y_combined = [area + power for area, power in zip(y_area, y_power)]

# Convert lists to numpy arrays
X = np.array(X)
y_combined = np.array(y_combined)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_combined, test_size=0.1, random_state=42)

# Create regression model and train
reg = LinearRegression().fit(X_train, y_train)

# Predict on the entire dataset (you can also use just the test set if preferred)
predicted_y = reg.predict(X)

# Calculate percentage differences
overall_diff = np.zeros_like(predicted_y[0])
for true_val, pred_val in zip(y_combined, predicted_y):
    percentage_diff = abs(pred_val - true_val) / (true_val + 1e-8)
    
    # Apply the >3 condition only for area data (first 6 entries)
    mask = np.concatenate([abs(pred_val[:6] - true_val[:6]) > 3, [1,1,1,1]])  # 1 is used as a mask for power data to always calculate difference
    overall_diff += percentage_diff * mask
    
    print("params: ", [f'{num}' for num in X[i]])
    print("predict: ", [f'{num:.3f}' for num in pred_val])
    print("gold: ", [f'{num:.3f}' for num in true_val])
    print('diff:', [f'{num:.3f}' for num in abs(pred_val-true_val)])
    print('==============')


overall_diff /= len(y_combined)

np.set_printoptions(precision=4, suppress=True, formatter={'float': '{: 0.4f}'.format})
print('Average percentage difference:', overall_diff)
#[latency,BRAM,DSP,FF,LUT,URAM],[TotalPower,FPGAPower,HBMPower,DynamicPower]]

params:  ['64', '64', '2', '4', '4', '2', '10', '10']
predict:  ['402.273', '0.000', '184.393', '138949.916', '93280.830', '0.000', '5.318', '5.075', '0.243', '2.146']
gold:  ['432.000', '0.000', '190.000', '156303.000', '96933.000', '0.000', '5.315', '5.072', '0.243', '2.143']
diff: ['29.727', '0.000', '5.607', '17353.084', '3652.170', '0.000', '0.003', '0.003', '0.000', '0.003']
params:  ['64', '64', '2', '4', '4', '2', '10', '10']
predict:  ['280.867', '0.000', '122.368', '72955.321', '56814.999', '0.000', '4.270', '4.028', '0.242', '1.119']
gold:  ['254.000', '0.000', '115.000', '55807.000', '52107.000', '0.000', '3.899', '3.658', '0.241', '0.756']
diff: ['26.867', '0.000', '7.368', '17148.321', '4707.999', '0.000', '0.371', '0.370', '0.001', '0.363']
params:  ['64', '64', '2', '4', '4', '2', '10', '10']
predict:  ['326.985', '0.000', '92.024', '30980.411', '40676.189', '0.000', '4.680', '4.438', '0.242', '1.520']
gold:  ['391.000', '0.000', '80.000', '53030.000', '38165.000', '0.0